# Using ROOT to bind Python and C++

## What is PyROOT?

* **PyROOT** is the name of the Python bindings offered by ROOT.
* All the ROOT C++ functions and classes are accessible from Python via PyROOT
  * Python façade, C++ performance
* But PyROOT is not just for ROOT!
  * It can also call into user-defined C++ code

## How does PyROOT work?

* PyROOT is a special type of bindings, since it's **automatic** and **dynamic**
 * No static wrapper generation
 * Dynamic python proxies are created for C++ entities
 * Lazy class/variable lookup
* Powered by [cppyy](https://cppyy.readthedocs.io/), the ROOT type system and [Cling](https://root.cern/cling/)
 * Reflection information
 * JIT C++ compilation and execution
* And on top of the automatic bindings: **pythonizations**
 * To make the use of C++ from Python simpler, more pythonic

## Using ROOT from Python

To start working with PyROOT, we need to import the ROOT module.

In [1]:
import ROOT

Welcome to JupyROOT 6.24/00


The ROOT Python module is the entry point for all the ROOT C++ functionality.

For example, we can create a histogram with ROOT using the `TH1F` C++ class from Python:

In [2]:
h = ROOT.TH1F("my_histo", "Example histogram", 100, -4, 4)

Now, imagine that the histogram were stored in a file and we wanted to retrieve it.

ROOT has its own file format that can store C++ objects. Let's open a ROOT file with the `TFile` class:

In [3]:
f = ROOT.TFile("data/example.root")

In order to get the histogram out of the file, we can use a pythonization: we access the histogram by its name as if it were an attribute of the file.

In [4]:
h = f.my_histo
print(h)

Name: my_histo Title: Example histogram NbinsX: 100


## Calling user-defined C++ code via PyROOT

We've seen how PyROOT allows to access all the functions and classes that the ROOT C++ libraries define.

In addition, it is possible to make PyROOT call into user-defined C++. For example, it is possible to declare a C++ function:

In [5]:
ROOT.gInterpreter.Declare("""
double add(double a, double b) {
    return a + b;
}
""")

True

and use it right away from Python:

In [6]:
ROOT.add(3.14, 100)

103.14

### What about code in C++ libraries?

In the example we just saw, the user-defined C++ code is contained in strings in our program, but PyROOT can also load and call into C++ libraries. This enables you to write high-performance C++, compile it and use it from Python.

More information can be found [here](https://root.cern/manual/python/#loading-user-libraries-and-just-in-time-compilation-jitting).

## Type conversions

When calling C++ from Python via PyROOT, there needs to be a conversion between the Python arguments we pass and the C++ arguments that the C++ side expects. PyROOT takes care of such conversion automatically, for example from Python integer to C++ integer:

In [7]:
ROOT.gInterpreter.Declare("void print_integer(int i) { std::cout << i << std::endl; }")

ROOT.print_integer(7)

7


Of course not every conversion is allowed!

In [8]:
ROOT.print_integer([]) # fails with TypeError

TypeError: void ::print_integer(int i) =>
    TypeError: could not convert argument 1 (int/long conversion expects an integer object)

As an example, you can convert from a NumPy array to a C-style array of the same type:

In [9]:
import numpy as np
a = np.array([1.,2.], dtype='float64')

ROOT.gInterpreter.Declare('''
void print_array(double *a) { std::cout << a[0] << "," << a[1] << std::endl; }
''')

ROOT.print_array(a)

1,2


or from a Python list to an `std::vector`:

In [10]:
ROOT.gInterpreter.Declare('''
void print_vector(std::vector<std::string> v) {
    for (auto &s : v) {
        std::cout << s << std::endl;
    }
}
''')

ROOT.print_vector(['Two', 'Words'])

Two
Words


## A final note on performance

Being able to call into C++ from Python does not guarantee that the performance of your Python script will always be the best, no matter what code you write!

In general, any heavy computation should be pushed to C++, e.g. encapsulating it in some C++ function that you call from Python.

In the context of high-energy physics, iterating over the collision events in a dataset is a common operation. Such iteration in Python can be slow for big datasets and should only be done during short exploratory work. Later in this course we will see how the event loop can be efficiently executed in C++, even from a Python script, with the help of ROOT's [RDataFrame](https://root.cern/doc/master/classROOT_1_1RDataFrame.html).

```python
# This can be slow!
for event in my_tree:
    h.Fill(event.some_branch)
```